In [1]:
import paddle
import paddlenlp
import pandas as pd
import numpy as np
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import f1_score, accuracy_score
from paddle.io import Dataset,DataLoader
from paddlenlp.trainer import Trainer, TrainingArguments
paddle.device.set_device('gpu:0')

Place(gpu:0)

In [2]:
data_train = pd.read_excel("/home/aistudio/work/评测数据集/train/usual_train.xlsx") #训练集
data_test=pd.read_excel("/home/aistudio/work/评测数据集/test（最终评测集）/真实评测集/usual_test_labeled.xlsx") #测试集
data_eval=pd.read_excel("/home/aistudio/work/评测数据集/eval（刷榜数据集）/usual_eval_labeled.xlsx") #验证集

#数据准备，训练用到的为训练集和测试集需要向量化
train_sentence=np.array(data_train["文本"])
train_label=np.array(data_train["情绪标签"])
eval_sentence=np.array(data_eval["文本"])
eval_label=np.array(data_eval["情绪标签"])

#自定义DataSet类
class DataSet(Dataset):
    def __init__(self, data, label, tokenizer):
        self.data = data
        self.label = label
        self.input_data = tokenizer(
            [sentence for sentence in data],
            max_seq_len=512, 
            pad_to_max_seq_len=True
        )

    def __len__(self):
        return len(self.label)

    def __getitem__(self, item):
        return {
            "input_ids": self.input_data['input_ids'][item],
            "token_type_ids": self.input_data['token_type_ids'][item],
            "labels": self.label[item],
        }

#导入预训练好的中文tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
#将训练集和验证集实例化
train_set=DataSet(train_sentence,train_label,tokenizer)
eval_set=DataSet(eval_sentence,eval_label,tokenizer)


[2024-03-04 12:49:11,078] [    INFO] - We are using <class 'paddlenlp.transformers.bert.tokenizer.BertTokenizer'> to load 'bert-base-chinese'.
[2024-03-04 12:49:11,085] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/bert-base-chinese/bert-base-chinese-vocab.txt
[2024-03-04 12:49:11,106] [    INFO] - tokenizer config file saved in /home/aistudio/.paddlenlp/models/bert-base-chinese/tokenizer_config.json
[2024-03-04 12:49:11,114] [    INFO] - Special tokens file saved in /home/aistudio/.paddlenlp/models/bert-base-chinese/special_tokens_map.json


In [3]:
#指定好训练参数
training_args = TrainingArguments(
    output_dir='./work/output',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,  
    dataloader_num_workers=2,
    logging_dir='./work/logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=10,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
)

[2024-03-04 12:49:24,318] [    INFO] - The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [4]:
#定义一个评价函数
def compute_metrics(pred):
    lables = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(lables, preds)
    f1 = f1_score(lables, preds, average="macro")
    return {
        "accuracy": acc,
        "f1": f1,
    }

In [5]:
#导入预训练的中文BERT模型
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_classes=6,cache_dir="./work/")
#指定所用模型，损失函数，训练参数，数据集，验证集，评价函数，然后实例化训练器
trainer=Trainer(
    model=model,
    criterion=paddle.nn.CrossEntropyLoss(),
    args=training_args,
    train_dataset=train_set,
    eval_dataset=eval_set,
    compute_metrics=compute_metrics
)

[2024-03-04 12:49:24,331] [    INFO] - We are using <class 'paddlenlp.transformers.bert.modeling.BertForSequenceClassification'> to load 'bert-base-chinese'.
[2024-03-04 12:49:24,335] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/bert-base-chinese/bert-base-chinese.pdparams
W0304 12:49:24.340519  3149 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.2
W0304 12:49:24.345371  3149 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
[2024-03-04 12:49:28,127] [    INFO] - ============================================================
[2024-03-04 12:49:28,137] [    INFO] -     Training Configuration Arguments    
[2024-03-04 12:49:28,143] [    INFO] - paddle commit id              :3fa7a736e32508e797616b6344d97814c37d3ff8
[2024-03-04 12:49:28,149] [    INFO] - _no_sync_in_gradient_accumulation:True
[2024-03-04 12:49:28,154] [    INFO] - adam_beta1                    :0.9
[2024-03-04 12:49:28,157] [    IN

In [6]:
#开始训练，训练完成后将模型保存再work目录下
trainer.train()
trainer.save_model("./work/model")

[2024-03-04 12:49:28,449] [    INFO] - ***** Running training *****
[2024-03-04 12:49:28,452] [    INFO] -   Num examples = 27766
[2024-03-04 12:49:28,455] [    INFO] -   Num Epochs = 10
[2024-03-04 12:49:28,458] [    INFO] -   Instantaneous batch size per device = 16
[2024-03-04 12:49:28,460] [    INFO] -   Total train batch size (w. parallel, distributed & accumulation) = 16
[2024-03-04 12:49:28,463] [    INFO] -   Gradient Accumulation steps = 1
[2024-03-04 12:49:28,466] [    INFO] -   Total optimization steps = 17360
[2024-03-04 12:49:28,469] [    INFO] -   Total num train samples = 277660


  0%|          | 0/17360 [00:00<?, ?it/s]

loss: 1.63693829, learning_rate: 4.997119815668203e-05, global_step: 10, interval_runtime: 6.1632, interval_samples_per_second: 2.596, interval_steps_per_second: 1.623, epoch: 0.0058
loss: 1.41821852, learning_rate: 4.994239631336406e-05, global_step: 20, interval_runtime: 4.3377, interval_samples_per_second: 3.689, interval_steps_per_second: 2.305, epoch: 0.0115
loss: 1.16201048, learning_rate: 4.991359447004609e-05, global_step: 30, interval_runtime: 4.3464, interval_samples_per_second: 3.681, interval_steps_per_second: 2.301, epoch: 0.0173
loss: 1.00052643, learning_rate: 4.988479262672811e-05, global_step: 40, interval_runtime: 4.3461, interval_samples_per_second: 3.681, interval_steps_per_second: 2.301, epoch: 0.023
loss: 0.9472127, learning_rate: 4.985599078341014e-05, global_step: 50, interval_runtime: 4.3423, interval_samples_per_second: 3.685, interval_steps_per_second: 2.303, epoch: 0.0288
loss: 1.01215458, learning_rate: 4.9827188940092165e-05, global_step: 60, interval_runt

[2024-03-04 12:50:57,556] [    INFO] - ***** Running Evaluation *****
[2024-03-04 12:50:57,558] [    INFO] -   Num examples = 2000
[2024-03-04 12:50:57,561] [    INFO] -   Pre device batch size = 16
[2024-03-04 12:50:57,563] [    INFO] -   Total Batch size = 16
[2024-03-04 12:50:57,565] [    INFO] -   Total prediction steps = 125


  0%|          | 0/125 [00:00<?, ?it/s]

...

  0%|          | 0/125 [00:00<?, ?it/s]

eval_loss: 1.8923460245132446, eval_accuracy: 0.7515, eval_f1: 0.7134921686802377, eval_runtime: 21.0876, eval_samples_per_second: 94.843, eval_steps_per_second: 5.928, epoch: 10.0


[2024-03-04 15:34:10,300] [    INFO] - Saving model checkpoint to ./work/output/checkpoint-17360
[2024-03-04 15:34:14,719] [    INFO] - Deleting older checkpoint [work/output/checkpoint-15600] due to args.save_total_limit
[2024-03-04 15:34:14,821] [    INFO] - 
Training completed. 

[2024-03-04 15:34:14,830] [    INFO] - Loading best model from ./work/output/checkpoint-2800 (score: 0.7386200102986379).


train_runtime: 9889.5561, train_samples_per_second: 28.076, train_steps_per_second: 1.755, train_loss: 0.24620741825558307, epoch: 10.0


[2024-03-04 15:34:18,037] [    INFO] - Saving model checkpoint to ./work/model
